In [4]:
import numpy as np
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
from numpyro.diagnostics import hpdi

import patsy
import seaborn as sns
import matplotlib.pyplot as plt
import arviz as az
from BayesBrain import models,datasim,glm

Make a data simulator

In [6]:
sims=datasim.simulate_poisson_grouped()
X_dsgn=sims[0]
Y=sims[1]

In [7]:
#Make BayesBrain class
# FIT WITH INTERACTIONS
mod2fitall = glm.PoissonGLMbayes()

mod2fitall.add_data(y=jnp.array(Y))

# Learn smoothness from data
mod2fitall.define_model(model='grouped_ard', basis_x_list=X_dsgn, S_list=None,
                          tensor_basis_list=None, S_tensor_list=None)

paramglm = {'fittype': 'vi', 'guide': 'normal', 'visteps': 30000, 'optimtype': 'scheduled'}
mod2fitall.fit(params=paramglm,  fit_intercept=True, cauchy=0.1)

#Got to fix post sampling checking to beta_i * lambda_ard[i]
mod2fitall.sample_posterior(5000).summarize_posterior(90).coeff_relevance()


100%|██████████| 10000/10000 [00:01<00:00, 6466.67it/s, init loss: 14807749099520.0000, avg. loss [9501-10000]: 3631.0207]  


In [8]:
print(mod2fitall.coef_keep)

{'beta_0': array([1, 1, 1, 1, 1]),
 'beta_1': array([0, 1, 1, 1, 1]),
 'intercept': 1,
 'lambda_ard': array([1, 1])}